In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import json 
import os
import time
from sklearn import model_selection
import importlib.util
spec = importlib.util.spec_from_file_location("Wide_and_Deep_model", "/content/drive/MyDrive/Colab Notebooks/Wide_and_Deep_model.py")
Wide_and_Deep_model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(Wide_and_Deep_model)
Wide_Deep = Wide_and_Deep_model.Wide_Deep

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("train on device: ", device)

train on device:  cuda:0


In [ ]:
riid_all = pd.DataFrame()

for dirname, _, filenames in os.walk('/content/drive/MyDrive/Fellowship.AI/P1_bandit/dataset/datas'):
    for filename in filenames:
        print(filename)
        data_path = os.path.join(dirname, filename)
        riid = pd.read_csv(data_path, usecols=['riid'])
        riid_all = pd.concat((riid_all, riid),axis=0)

vocab = {r:i for i, r in enumerate(riid_all.riid.unique())}
print("number of total data points:", len(riid_all))
print("number of unique users:", len(vocab))
del riid, riid_all

sends_2020_wk01.csv
sends_2020_wk04.csv
sends_2020_wk02.csv
sends_2020_wk03.csv
sends_2020_wk00.csv
sends_2020_wk05.csv
sends_2020_wk07.csv
sends_2020_wk08.csv
sends_2020_wk06.csv
sends_2020_wk09.csv
sends_2020_wk12.csv
sends_2020_wk10.csv
sends_2020_wk11.csv
sends_2020_wk16.csv
sends_2020_wk14.csv
sends_2020_wk13.csv
sends_2020_wk17.csv
sends_2020_wk15.csv
sends_2020_wk19.csv
sends_2020_wk21.csv
sends_2020_wk18.csv
sends_2020_wk20.csv
sends_2020_wk22.csv
sends_2020_wk23.csv
sends_2020_wk24.csv
sends_2020_wk25.csv
sends_2019_wk27.csv
sends_2019_wk29.csv
sends_2019_wk28.csv
sends_2019_wk26.csv
sends_2019_wk33.csv
sends_2019_wk32.csv
sends_2019_wk30.csv
sends_2019_wk31.csv
sends_2019_wk34.csv
sends_2019_wk36.csv
sends_2019_wk37.csv
sends_2019_wk35.csv
sends_2019_wk41.csv
sends_2019_wk38.csv
sends_2019_wk40.csv
sends_2019_wk39.csv
sends_2019_wk43.csv
sends_2019_wk44.csv
sends_2019_wk42.csv
sends_2019_wk45.csv
sends_2019_wk46.csv
sends_2019_wk50.csv
sends_2019_wk49.csv
sends_2019_wk48.csv


In [ ]:
context_columns = ['sends_since_last_open',
                   'retention_score', 
                   'recency_score',
                   'frequency_score',
                   'is_one_for_free', 
                   'is_exclusive', 
                   'free_shipping',
                   'has_urgency', 
                   'sl_contains_price', 
                   'is_discount_mentioned',
                   'campaign_Brand', 
                   'campaign_Core', 
                   'campaign_Dedicated', 
                   'campaign_InnovationSpotlight',
                   'campaign_NewArrivals', 
                   'campaign_ProductSpotlight', 
                   'campaign_Replen',
                   'campaign_Tops', 
                   'campaign_Trend', 
                   'campaign_Other']

In [ ]:
# A custumized class of torch dataset for the training
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, df_data):
        self.data = np.array(df_data)

    def __getitem__(self, index):
        context = self.data[index][:-1]
        optimal_action = self.data[index][-1]
        return context, optimal_action

    def __len__(self):
        return self.data.shape[0]

In [ ]:
def train_week(device, model, optimizer, loss_func, dataloader, num_epoch=1):
    for i in range(num_epoch):
        model.train()
        corrects = 0.0
        train_loss = 0.0
        for c_train, opt_a_train in dataloader:
            c_train = c_train.float().to(device)
            opt_a_train = opt_a_train.long().to(device)

            pred_out_train = model(c_train)

            loss = loss_func(pred_out_train, opt_a_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pred_a_train = torch.argmax(pred_out_train, dim=1)
            corrects += torch.sum(pred_a_train==opt_a_train).item()
            train_loss += loss.item() * c_train.shape[0]
    return corrects, train_loss

def valid(device, model, loss_func, dataloader):
    model.eval()
    corrects = 0.0
    losses = 0.0
    for c, opt_a in dataloader:
        c = c.float().to(device)
        opt_a = opt_a.long().to(device)
        pred_out = model(c)
        loss = loss_func(pred_out, opt_a)
        pred_a = torch.argmax(pred_out, dim=1)
        corrects += torch.sum(pred_a==opt_a).item()
        losses += loss.item() * c.shape[0]
    accuracy = corrects/len(dataloader.dataset)
    average_loss = losses/len(dataloader.dataset)
    return accuracy, average_loss

In [ ]:
# create a deep-only model and train it
wide_dim = 1
deep_dim = 20
action_dim = 2
embeddings = {'wide':[['user_id', len(vocab), 64]]}
deep_neurons = [128, 64]
model = Wide_Deep(wide_dim, deep_dim, action_dim, embeddings=embeddings, deep_neurons=deep_neurons).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

This is a wide and deep model.


In [ ]:
num_epoch = 3
for i in range(num_epoch):
    start_time = time.time()
    if i==0:
        df_valid = pd.DataFrame()
    total_corrects = 0.0
    total_loss = 0.0
    total_len = 0
    for dirname, _, filenames in os.walk('/content/drive/MyDrive/Fellowship.AI/P1_bandit/dataset/datas'):
        for filename in filenames:
            print(filename)
            data_path = os.path.join(dirname, filename)
            df_week = pd.read_csv(data_path, usecols=['riid']+context_columns+['optimal_action'])
            df_week['riid'] = df_week.riid.apply(lambda x:vocab[x])
            df_week_train, df_week_valid = model_selection.train_test_split(df_week, train_size=0.95, random_state=33)
            if i==0:
                df_valid = pd.concat((df_valid, df_week_valid), axis=0)
            batch_size = 8192
            dataset = Mydataset(df_week_train)
            dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
            corrects_week, loss_week = train_week(device, model, optimizer, loss_func, dataloader)
            total_corrects += corrects_week
            total_loss += loss_week
            total_len += len(dataset)

    train_accuracy = total_corrects/total_len
    train_average_loss = total_loss/total_len
    print("Epoch:{:>4}\t Training Loss: {:6.3f}\t Training Accuracy: {:.3f}\t time used: {:.3f} sec".format(i+1, train_average_loss, train_accuracy, time.time()-start_time))
    start_time = time.time()
    dataset = Mydataset(df_valid)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    valid_accuracy, valid_average_loss = valid(device, model, loss_func, dataloader)
    print("Epoch:{:>4}\t Valid Loss: {:6.3f}\t Valid Accuracy: {:.3f}\t time used: {:.3f} sec".format(i+1, valid_average_loss, valid_accuracy, time.time()-start_time))


sends_2020_wk01.csv
sends_2020_wk04.csv
sends_2020_wk02.csv
sends_2020_wk03.csv
sends_2020_wk00.csv
sends_2020_wk05.csv
sends_2020_wk07.csv
sends_2020_wk08.csv
sends_2020_wk06.csv
sends_2020_wk09.csv
sends_2020_wk12.csv
sends_2020_wk10.csv
sends_2020_wk11.csv
sends_2020_wk16.csv
sends_2020_wk14.csv
sends_2020_wk13.csv
sends_2020_wk17.csv
sends_2020_wk15.csv
sends_2020_wk19.csv
sends_2020_wk21.csv
sends_2020_wk18.csv
sends_2020_wk20.csv
sends_2020_wk22.csv
sends_2020_wk23.csv
sends_2020_wk24.csv
sends_2020_wk25.csv
sends_2019_wk27.csv
sends_2019_wk29.csv
sends_2019_wk28.csv
sends_2019_wk26.csv
sends_2019_wk33.csv
sends_2019_wk32.csv
sends_2019_wk30.csv
sends_2019_wk31.csv
sends_2019_wk34.csv
sends_2019_wk36.csv
sends_2019_wk37.csv
sends_2019_wk35.csv
sends_2019_wk41.csv
sends_2019_wk38.csv
sends_2019_wk40.csv
sends_2019_wk39.csv
sends_2019_wk43.csv
sends_2019_wk44.csv
sends_2019_wk42.csv
sends_2019_wk45.csv
sends_2019_wk46.csv
sends_2019_wk50.csv
sends_2019_wk49.csv
sends_2019_wk48.csv
